In [51]:
from torchvision import models
import torch.nn as nn
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable as V
from torchvision import transforms as trn
import os
import pickle
from PIL import Image

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def dump_data(data, filename):
    print('writing file: ' + filename)
    with open(filename, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        
def load_data(file):
   
    print('loading file: ' + file)
    with open(file, 'rb') as f:
        data = pickle.load(f)

    return(data)

# =============================================================================
# Configuration
# =============================================================================
# Set paths and options
PRETRAINED = True  # Use pretrained AlexNet model
PROJECT_DIR = "/projects/crunchie/boyanova/EEG_Things/"


# Set random seed for reproducibility
seed = 20200220
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.use_deterministic_algorithms(True)

# =============================================================================
# Define AlexNet model class with layer extraction
# =============================================================================
# Define convolutional and fully connected layers of interest
conv_layers = ['conv1', 'ReLU1', 'maxpool1', 'conv2', 'ReLU2', 'maxpool2',
               'conv3', 'ReLU3', 'conv4', 'ReLU4', 'conv5', 'ReLU5', 'maxpool5']
fully_connected_layers = ['Dropout6', 'fc6', 'ReLU6', 'Dropout7', 'fc7', 'ReLU7', 'fc8']

class AlexNet(nn.Module):
    def __init__(self, pretrained=True):
        """Select the desired layers and create the model."""
        super(AlexNet, self).__init__()
        self.select_cov = ['conv1', 'ReLU1', 'maxpool1', 'conv2', 'ReLU2', 'maxpool2']
        self.select_fully_connected = ['fc7', 'ReLU7', 'fc8']
        self.feat_list = self.select_cov + self.select_fully_connected
        self.alex_feats = models.alexnet(pretrained=pretrained).features
        self.alex_classifier = models.alexnet(pretrained=pretrained).classifier
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

    def forward(self, x):
        """Extract the feature maps."""
        features = []
        for name, layer in self.alex_feats._modules.items():
            x = layer(x)
            if conv_layers[int(name)] in self.feat_list:
                features.append(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        for name, layer in self.alex_classifier._modules.items():
            x = layer(x)
            if fully_connected_layers[int(name)] in self.feat_list:
                features.append(x)
        return features

# Instantiate the model
model = AlexNet(pretrained=PRETRAINED)
if torch.cuda.is_available():
    model.cuda()
model.eval()

# =============================================================================
# Define image preprocessing
# =============================================================================
centre_crop = trn.Compose([
    trn.Resize((224, 224)),
    trn.ToTensor(),
    trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# =============================================================================
# Process images and extract feature maps
# =============================================================================
# Directory to save extracted feature maps
save_dir = os.path.join(PROJECT_DIR, 'eeg_experiment','model_activations', 'alexnet')
os.makedirs(save_dir, exist_ok=True)

# Get image list from specified directory

experimental_stimuli = pd.read_csv(os.path.join(PROJECT_DIR, "eeg_prep/scripts/exp_stimuli_desc.csv"))
experimental_stimuli = experimental_stimuli["stim_name"].values
image_list = [os.path.join(PROJECT_DIR, "eeg_prep", "stimuli_shined", x) for x in experimental_stimuli]

# Extract and save feature maps
for i, image_path in enumerate(image_list):
    img = Image.open(image_path).convert('RGB')
    input_img = V(centre_crop(img).unsqueeze(0))
    if torch.cuda.is_available():
        input_img = input_img.cuda()
    
    # Forward pass to extract features
    feature_maps = model.forward(input_img)
    feats = {model.feat_list[f]: feature_map.data.cpu().numpy() for f, feature_map in enumerate(feature_maps)}
    
    # Save feature maps
    file_name = experimental_stimuli[i].split(".")[0] + ".pickle"
    dump_data(feats, os.path.join(save_dir, file_name))

print("Feature extraction completed.")

# =============================================================================
# Standardize feature maps
# =============================================================================

folder_path = save_dir
stimulus_feature_dict = {}


standardized_maps = dict()
for key in feats.keys():
    print(key)
    all_feature_maps = []
    stimulus_names = []

     
    for filename in os.listdir(folder_path):
        if filename.endswith(".pickle"):  # Assuming feature maps are saved in pickle files
            file_path = os.path.join(folder_path, filename)
            feature_map_dict = load_data(file_path)

            # Extract the stimulus name (assuming it's the first key or a known key)
            # Adjust 'stimulus_name' if the key is different
            stimulus_name = feature_map_dict.get('stimulus_name') or list(feature_map_dict.keys())[0]
            stimulus_names.append(filename.split(".")[0])

            # Convert the rest of the feature map data to an array (exclude the name if needed)
            feature_map_array = feature_map_dict[key].flatten()# Skip name if it's in first entry
            all_feature_maps.append(feature_map_array)
            concatenated_features = np.array(all_feature_maps)

    # Apply StandardScaler to standardize the data
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(concatenated_features)
    standardized_maps[key] = concatenated_features
    del scaled_features

standardized_maps["stimulus_names"] = stimulus_names 
dump_data(standardized_maps, os.path.join(save_dir, "standardized_maps.pickle"))
print("Feature group standarization complete!")

/projects/crunchie/boyanova/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/projects/crunchie/boyanova/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


writing file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/model_activations/alexnet/tomato_11s.pickle
writing file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/model_activations/alexnet/chandelier_10s.pickle
writing file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/model_activations/alexnet/blazer_08s.pickle
writing file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/model_activations/alexnet/dustpan_01b.pickle
Feature extraction completed.
conv1
loading file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/model_activations/alexnet/tomato_11s.pickle
loading file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/model_activations/alexnet/chandelier_10s.pickle
loading file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/model_activations/alexnet/blazer_08s.pickle
loading file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/model_activations/alexnet/dustpan_01b.pickle
ReLU1
loading file: /projects/crunchie/boyanova/EEG_Things